<a href="https://colab.research.google.com/github/MuhammedShiway/TEKNOFEST-2023/blob/main/EXPLORE_PROBLEMATIC_DICOM_IMAGES_%5BCOMPLETE%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydicom opencv-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pydicom
import numpy as np
import cv2
from google.colab import drive
import time
import matplotlib.pyplot as plt
from PIL import Image



In [ ]:
# Mount Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
folder_path = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/TEKNOFEST_MG_EGITIM_1"
png1024_path = "/content/gdrive/My Drive/Colab Notebooks/TÜSEB 2023/PNG1024"


In [ ]:
def crop_image(img, show=False):
    # Binarize the image
    bin_pixels = cv2.threshold(img, 20, 255, cv2.THRESH_BINARY)[1]
   
    # Make contours around the binarized image, keep only the largest contour
    contours, _ = cv2.findContours(bin_pixels, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contour = max(contours, key=cv2.contourArea)

    # Create a mask from the largest contour
    mask = np.zeros(img.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED)
   
    # Use bitwise_and to get masked part of the original image
    out = cv2.bitwise_and(img, mask)
    
    # get bounding box of contour
    y1, y2 = np.min(contour[:, :, 1]), np.max(contour[:, :, 1])
    x1, x2 = np.min(contour[:, :, 0]), np.max(contour[:, :, 0])
    
    x1 = int(0.99 * x1)
    x2 = int(1.01 * x2)
    y1 = int(0.99 * y1)
    y2 = int(1.01 * y2)
    
    if show:
        plt.imshow(out[y1:y2, x1:x2], cmap="gray") ; 

    return out[y1:y2, x1:x2]

In [ ]:
image_size = 1024

In [ ]:

def convert_dicom_to_png(dicom_folder_path, png_folder_path):
    """
    Converts DICOM images in a folder to PNG images and saves them to a new folder.
    Checks if PNG images for the folder already exist and creates the new folder if necessary.
    """
    # Check if the PNG image folder for the DICOM folder already exists
    dicom_folder_name = os.path.basename(dicom_folder_path)
    png_folder = os.path.join(png_folder_path, dicom_folder_name)
    if os.path.exists(png_folder):
        return
    else:
        os.makedirs(png_folder, exist_ok=True)

    # Convert the DICOM images to PNG and save them to the new folder
    for dicom_file in os.listdir(dicom_folder_path):
        dicom_file_path = os.path.join(dicom_folder_path, dicom_file)

        # Check if the file is a directory
        if os.path.isdir(dicom_file_path):
            continue

        # Load the DICOM image
        ds = pydicom.dcmread(dicom_file_path)
        ###########################################
        # Get pixel data
        pixel_data = ds.pixel_array

        #normalize to range [0, 255]
        p_data = (pixel_data - np.min(pixel_data)) / (np.max(pixel_data) - np.min(pixel_data))
        p_data = (p_data * 255).astype(np.uint8)

        #resize to desired size
        res_img = cv2.resize(p_data, (image_size, image_size))

        img_mean = res_img.mean()
        if img_mean > 125:
          res_img = 255 - res_img
        else:
          res_img = res_img
        
        #SPECIAL CONDITIONS
        if '822670571' in dicom_file_path:
          if 'RMLO' in dicom_file_path:
            res_img = res_img[:, 400:1024]
          elif 'RCC' in dicom_file_path:
            res_img = res_img[:, 400:1024]
          elif 'LMLO' in dicom_file_path:
            res_img = res_img[:, 0:600]
          elif 'LCC' in dicom_file_path:
            res_img = res_img[:, 0:600]

        elif '822688070' in dicom_file_path:
          if 'RMLO' in dicom_file_path:
            res_img = np.fliplr(res_img)
          elif 'RCC' in dicom_file_path:
            res_img = np.fliplr(res_img)
          elif 'LMLO' in dicom_file_path:
            res_img = np.fliplr(res_img)

        elif '845283998' in dicom_file_path:
          if 'RMLO' in dicom_file_path:
            res_img = res_img[:, 500:1024]
        
        elif '845284962' in dicom_file_path:
          if 'LCC' in dicom_file_path:
            res_img = res_img[:, 0:520]
          elif 'RCC' in dicom_file_path:
            res_img = res_img[:, 560:1024]

        #If nothing else
        else:
          res_img = res_img
  
            

        cropped = crop_image(res_img, False)
        # Resize the image to 1024x1024
        region = cv2.resize(cropped, (image_size, image_size))

        # Convert pixel data to image and save as PNG

        png_file_path = os.path.join(png_folder, dicom_file.replace(".dcm", ".png"))
        cv2.imwrite(png_file_path, region)

        #########################################

        # Delete them from memory
        del ds
        del res_img
        del region

start_time = time.time()
#Loop through each sub-folder in the DICOM folder
for sub_folder in os.listdir(folder_path):
    if sub_folder.endswith(".xlsx"):
        continue
    
    sub_folder_path = os.path.join(folder_path, sub_folder)
    
    # Check if the folder already exists in the PNG folder
    png_folder = os.path.join(png1024_path, sub_folder)
    if not os.path.exists(png_folder):
        # Convert DICOM images to PNG and save them to the new folder
        convert_dicom_to_png(sub_folder_path, png1024_path)

end_time = time.time()
print("Total time taken:", end_time - start_time, "seconds")


Total time taken: 9933.889211416245 seconds
